# パスワードとハッシュ化について考えてみた

- こういうの見たことない？

<img src="image/スクリーンショット 2025-09-02 224647.png">


## 平文パスワード

- こういうやつ

| user_name | password |
|-----------|----------|
| user1     | 12345    |
| user2     | password |
| user3     | 12345    |

※「パスワードがそのまま」データベースに格納されている


## 何が問題？
- データベースが侵害された場合、パスワードがそのまま漏洩する
- 漏洩したサービスのアカウントが不正ログインされる
- 更に他のサービスもアカウントが不正ログインされる
  - ユーザーはパスワードを使いまわす

## ハッシュ化
- パスワードをハッシュ化して保存する
- 入力されたパスワードを同じようにハッシュ化して、保存されているハッシュ値と比較する

| user_name | password_hash                    |
|-----------|----------------------------------|
| user1     | 827ccb0eea8a706c4c34a16891f84e7b |
| user2     | 5f4dcc3b5aa765d61d8327deb882cf99 |

※「827ccb0eea8a706c4c34a16891f84e7b」は「12345」のハッシュ値  
※「5f4dcc3b5aa765d61d8327deb882cf99」は「password」のハッシュ値

1. 入力された値
    - user_name: user1
    - password: 12345    
2. パスワードをハッシュ化すると「827ccb0eea8a706c4c34a16891f84e7b」
3. ハッシュ値を比較すると同じ値なのでパスワードが一致している！！


## ハッシュ値から元のパスワードを推測することはできない
- 「827ccb0eea8a706c4c34a16891f84e7b」から「12345」を推測することはできたら意味がない
- 一度変換してしまったら元に戻す方法が存在しないから

この辺りが、パスワード教えてという問い合わせに回答することができない理由  
パスワードを保持していない + ハッシュからパスワードを推測することができない  
→ パスワードはわからない

※なので、パスワードを教えてくれるサービスはちょっと怖い

## 暗号化とハッシュ化の違い
- 暗号化は元の文字列を復元できる
- ハッシュ化は元の文字列を復元できない

※復元できないって意味がわからなくない？？（何回でも同じハッシュ値が出力されるのに）

## ハッシュ化は圧縮

- たとえば「数字を 100 で割った余り」を返す仕組み
  - 12345 → 45
  - 23445 → 45
  - 34545 → 45

「45」から元の数値に戻すことができない

このようなことをハッシュ化という。
ここで大事なことは、「1つのハッシュ値に対して元の文字列が複数存在する」ということ。

## 正しいパスワード以外でログインできる？
- ログインできる
- 仮に「a」のハッシュ値が「12345」で、「b」のハッシュ値が「12345」だったとするとログイン可能


## 現実的には起こりえない
- 衝突する別のパスワードを探すのは、宇宙の寿命が終わるまで計算しても無理レベル
- そもそもハッシュ値が衝突する確率は極めて低い
  - ハッシュ値が40桁の場合
    - 1000人が1日に10回試すとする
    - 4×10の17乗年間続けると、50%の確率で衝突が発生する
  - 4×10の17乗年間とは宇宙の年齢よりも長い

よって、「衝突がある」ことは理論的に避けられないけど、現実的には問題にならない

## 話しを戻す

## ハッシュ化すると元のパスワードがわからない
- 実際に試して同じになるまで試すしかない


## データベースが漏洩してもパスワードがわからない
- データベースにはハッシュ値しかない
- 漏洩しても元のパスワードがわからない

## 「漏洩しても安心」ではなく「漏洩してもすぐに危険にはならない」
- 漏洩したとしても、すぐに危険になるわけではない
- その間にユーザーへ「パスワードを変更してください」と通知できる
- つまり、「被害を食い止める時間を稼げる仕組み」がハッシュ化

※ハッシュ化は保険

## ハッシュ化にも種類（アルゴリズム）がある
- それぞれ特徴や強さが違う
- 昔は「MD5」や「SHA-1」という方式がよく使われていた
- でも今は「弱いハッシュ」とされ、セキュリティ上は絶対使ってはいけない


## 弱いハッシュ？
- 衝突するハッシュを簡単に作り出せる
- 計算が早すぎる
  - ハッシュ化の計算コストが低いので、パスワードを総当たりで解析する攻撃に弱い
- そもそも、レインボーテーブルという対応表が公開されている
  - 「よく使われるパスワード → MD5 ハッシュ」
  - 例えば「827ccb0eea8a706c4c34a16891f84e7b」があったら、一瞬で「12345」と突き止められる。

## 実際に試してみる


In [8]:
import bcrypt # pip install bcrypt
import hashlib

In [ ]:
# 弱いハッシュの代表格「MD5」
text1 = "12345"
hash1 = hashlib.md5(text1.encode()).hexdigest()
print(f"text1: {text1}, hash1: {hash1}")

text2 = "password"
hash2 = hashlib.md5(text2.encode()).hexdigest()
print(f"text2: {text2}, hash2: {hash2}")


text1: 12345, hash1: 827ccb0eea8a706c4c34a16891f84e7b
text2: password, hash2: 5f4dcc3b5aa765d61d8327deb882cf99


In [11]:
# 弱いハッシュの代表格「MD5」
text3 = "TEXTCOLLBYfGiJUETHQ4hAcKSMd5zYpgqf1YRDhkmxHkhPWptrkoyz28wnI9V0aHeAuaKnak"
text4 = "TEXTCOLLBYfGiJUETHQ4hEcKSMd5zYpgqf1YRDhkmxHkhPWptrkoyz28wnI9V0aHeAuaKnak"
#                             ↑22文字目が「A」と「E」で異なる

hash3 = hashlib.md5(text3.encode()).hexdigest()
print(f"text3: {text3}, hash3: {hash3}")

hash4 = hashlib.md5(text4.encode()).hexdigest()
print(f"text4: {text4}, hash4: {hash4}")

text3: TEXTCOLLBYfGiJUETHQ4hAcKSMd5zYpgqf1YRDhkmxHkhPWptrkoyz28wnI9V0aHeAuaKnak, hash3: faad49866e9498fc1719f5289e7a0269
text4: TEXTCOLLBYfGiJUETHQ4hEcKSMd5zYpgqf1YRDhkmxHkhPWptrkoyz28wnI9V0aHeAuaKnak, hash4: faad49866e9498fc1719f5289e7a0269


```
text3: TEXTCOLLBYfGiJUETHQ4hAcKSMd5zYpgqf1YRDhkmxHkhPWptrkoyz28wnI9V0aHeAuaKnak, hash3: faad49866e9498fc1719f5289e7a0269
text4: TEXTCOLLBYfGiJUETHQ4hEcKSMd5zYpgqf1YRDhkmxHkhPWptrkoyz28wnI9V0aHeAuaKnak, hash4: faad49866e9498fc1719f5289e7a0269
````

↑っと出ているはずで、ハッシュの衝突が確認できる  
つまり、MD5でハッシュ化しているサービスがあるとすると、↑の2つのパスワードどちらでもログインが可能

※簡単に出しているけどこの例はかなり例外で、この短さでの衝突はこれ以外の文字列はパッと出てこない

## 実際にはハッシュから元のパスワードを推測される
- 漏洩したハッシュ「827ccb0eea8a706c4c34a16891f84e7b」から元のパスワードを推測することをひたすら試す

In [16]:
hash = "827ccb0eea8a706c4c34a16891f84e7b"

def f(s):
     return hashlib.md5(s.encode()).hexdigest()

# ありそうなパスワード
password_list = ["password", "abcde", "password1234", "12345", "qwerty"]

for password in password_list:
    if f(password) == hash:
        print(f"🤪パスワードは「{password}」です")
        break
    else:
        print(f"😃パスワードは「{password}」ではありません")



😃パスワードは「password」ではありません
😃パスワードは「abcde」ではありません
😃パスワードは「password1234」ではありません
🤪パスワードは「12345」です


## 弱いハッシュのところで「計算が早すぎる」という話しはここに繋がる
- ハッシュ化の計算が早いので、パスワードを総当たりで解析される
- ハッシュ化の計算が遅いと、総当たりで解析するのが難しい（時間がかかる = 時間が稼げる）

## ハッシュ化だけの場合に発生すること その1

| user_name | password_hash                    |
|-----------|----------------------------------|
| user1     | 827ccb0eea8a706c4c34a16891f84e7b |
| user2     | 5e884898da28047151d0e56f8dc62927 |
| user3     | 827ccb0eea8a706c4c34a16891f84e7b |

- user1とuser3は同じパスワードを使っていることがわかってしまう
- もしuser1のパスワードが漏洩した場合、user3も同じパスワードを使っているので、user3も不正ログインされる

## ソルト（塩）を使用する
- パスワードにソルトを加えてハッシュ化する
- ソルトはユーザーごとに異なる値

| user_name | password_hash                    | salt |
|-----------|----------------------------------|------|
| user1     | e8269a9e19201d84d0d3027720bf698a | 1234 |
| user2     | 1fb5d7da9191c7fb675d4bd0d4975fd4 | 5678 |
| user3     | a047343bf4ba65fd4c4ef9596c92960c | abcd |

- ハッシュ化する時に パスワード + ソルト をハッシュ化する
  - user1の場合、パスワードは「12345」でソルトは「1234」なので、「123451234」をハッシュ化する
  - user2の場合、パスワードは「12345」でソルトは「5678」なので、「123455678」をハッシュ化する
  - user3の場合、パスワードは「12345」でソルトは「abcd」なので、「12345abcd」をハッシュ化する
- 生成されるハッシュは異なる値となる

## ハッシュ化だけの場合に発生すること その2
- 頑張ればハッシュ値から元のパスワードを推測できるかもしれない

※計算速度は向上している

## ストレッチする
- ハッシュ化した値を更にハッシュ化する
- → ハッシュ生成に時間かかるようにすれば良い
- → ハッシュ生成に時間かかるようにすると、総当たりで解析するのが難しい

※時間がかかるとは言っても1件をハッシュ化するのであれば大した話ではない。総当たりで計算することを想定している


## ハッシュ化だけの場合に発生すること その3
- 頑張ればハッシュ値から元のパスワードを推測できるかもしれない
- ハッシュ値、ソルト、ストレッチ全部わかっているのであれば頑張れば推測できるかもしれない


## ペッパー
- データベースとは別の場所に文字列を保存しておく
- パスワード + ソルト + ペッパー でハッシュ値とする

※データベースが漏洩してもパスワードは漏洩しない

## まとめ
- パスワードを平文で保存するとデータベースが漏洩するとパスワードが漏洩する
- ハッシュ化するとパスワードが漏洩してもハッシュ値から元のパスワードを推測できない
- ハッシュ化とはデータベースが漏洩時の被害を軽減するための仕組み
- ハッシュ化に加えてソルトやストレッチ、ペッパーといった追加の保険もある

## おまけ
- 以前エンジニアリング勉強で話した資料は↓
  - https://github.com/yamap55/HashPlayground/blob/main/hash_playground.ipynb